In [1]:
from tqdm import tqdm
import datetime
from torch.utils.data import DataLoader
from alexnet import AlexNet
import torch.nn as nn
import torch.optim as optim


In [2]:
import torch
print("MPS available:", torch.backends.mps.is_available())

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

MPS available: True


In [3]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize(224),  # Resize images to fit AlexNet input dimensions.
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalization parameters for ImageNet.
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


/Users/luca/.local/share/virtualenvs/i2dl-P1n68_k-/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
Files already downloaded and verified


In [4]:
# loader runner.py and enable auto-reloading
%load_ext autoreload
%autoreload 2
from runner import run
from visualize import write_feature_maps

In [9]:
# Do AlexNet


hparams = {
    "lr": 1e-4,
    "num_classes": 10,
    "batch_size": 64,
    "epochs": 10
}

alexnet = AlexNet(hparams["num_classes"]).to(device)

train_loader = DataLoader(trainset, batch_size=hparams["batch_size"] , shuffle=True)
val_loader = DataLoader(testset, batch_size=hparams["batch_size"], shuffle=True)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=hparams["lr"])


# Train the model
run(hparams, alexnet, "alexnet-cifar10", criterion, optimizer, val_loader, train_loader, device)


Epoch 1:   2%|▏         | 12/782 [00:04<04:20,  2.96batch/s, loss=2.26, val_loss=0, val_acc=0]


KeyboardInterrupt: 

In [ ]:
write_feature_maps(alexnet, val_loader, device)

In [5]:
# train resnet
from resnet import ResNet

hparams = {
    "lr": 1e-4,
    "num_classes": 10,
    "batch_size": 64,
    "epochs": 10
}

resnet = ResNet(hparams["num_classes"]).to(device)

# print number of parameters
print("Number of parameters in ResNet:", sum(p.numel() for p in resnet.parameters() if p.requires_grad))

train_loader = DataLoader(trainset, batch_size=hparams["batch_size"] , shuffle=True)
val_loader = DataLoader(testset, batch_size=hparams["batch_size"], shuffle=True)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=hparams["lr"])

# Train the model
run(hparams, resnet, "resnet-cifar10", criterion, optimizer, val_loader, train_loader, device)
                                                       

Number of parameters in ResNet: 1227594


Epoch 1:  62%|██████▏   | 488/782 [1:51:10<13:57,  2.85s/batch, loss=1.77, val_loss=1.9, val_acc=tensor(0.3389, device='mps:0')]   